In [ ]:
import torch
import sys

print("=" * 60)
print("ПРОВЕРКА УСТАНОВКИ PYTORCH")
print("=" * 60)

print(f"Python version: {sys.version}")
print(f"PyTorch version: {torch.__version__}")
print(f"PyTorch CUDA version: {torch.version.cuda}")
print(f"CUDA available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    device = torch.device('cuda')
    print(f"Device: {device}")
    print(f"GPU count: {torch.cuda.device_count()}")
    print(f"GPU name: {torch.cuda.get_device_name(0)}")

    # Проверка памяти GPU
    print(f"GPU memory total: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")
    print(f"GPU memory allocated: {torch.cuda.memory_allocated(0) / 1024**3:.2f} GB")
    print(f"GPU memory reserved: {torch.cuda.memory_reserved(0) / 1024**3:.2f} GB")
else:
    device = torch.device('cpu')
    print(f"Device: {device} (CUDA не доступна!)")

print("=" * 60)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np

# Проверка GPU
print("=" * 60)
print("ИНФОРМАЦИЯ О СИСТЕМЕ")
print("=" * 60)
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    device = torch.device('cuda')
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")
    print(f"CUDA Version: {torch.version.cuda}")
else:
    device = torch.device('cpu')
    print("Используется CPU")

print("=" * 60)

# Создаем более сложную модель для демонстрации
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)
        self.dropout = nn.Dropout(0.25)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.pool(x)
        x = self.relu(self.conv2(x))
        x = self.pool(x)
        x = x.view(-1, 64 * 7 * 7)
        x = self.dropout(x)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Загружаем данные
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Скачиваем MNIST
print("Загрузка данных MNIST...")
train_dataset = torchvision.datasets.MNIST(
    root='./data',
    train=True,
    download=True,
    transform=transform
)

test_dataset = torchvision.datasets.MNIST(
    root='./data',
    train=False,
    download=True,
    transform=transform
)

# Создаем DataLoader с учетом GPU
batch_size = 128  # Можете увеличить, т.к. у вас 16GB памяти
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

print(f"Размер батча: {batch_size}")
print(f"Обучающих примеров: {len(train_dataset)}")
print(f"Тестовых примеров: {len(test_dataset)}")

# Создаем модель и перемещаем на GPU
model = CNNModel().to(device)
print(f"\nМодель создана и перемещена на {device}")

# Функция для подсчета параметров
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"Количество обучаемых параметров: {count_parameters(model):,}")

# Оптимизатор и функция потерь
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Функция обучения
def train_epoch(model, loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for batch_idx, (data, target) in enumerate(loader):
        # Перемещаем данные на GPU
        data, target = data.to(device), target.to(device)

        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = output.max(1)
        total += target.size(0)
        correct += predicted.eq(target).sum().item()

        if batch_idx % 100 == 0:
            print(f'  Batch {batch_idx}/{len(loader)}, Loss: {loss.item():.4f}')

    accuracy = 100. * correct / total
    avg_loss = running_loss / len(loader)
    return avg_loss, accuracy

# Функция валидации
def validate(model, loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for data, target in loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            loss = criterion(output, target)

            running_loss += loss.item()
            _, predicted = output.max(1)
            total += target.size(0)
            correct += predicted.eq(target).sum().item()

    accuracy = 100. * correct / total
    avg_loss = running_loss / len(loader)
    return avg_loss, accuracy

# Обучение модели
print("\n" + "=" * 60)
print("НАЧАЛО ОБУЧЕНИЯ")
print("=" * 60)

epochs = 5
train_losses, train_accs = [], []
val_losses, val_accs = [], []

for epoch in range(epochs):
    print(f'\nЭпоха {epoch+1}/{epochs}')

    # Обучение
    train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device)
    train_losses.append(train_loss)
    train_accs.append(train_acc)

    # Валидация
    val_loss, val_acc = validate(model, test_loader, criterion, device)
    val_losses.append(val_loss)
    val_accs.append(val_acc)

    print(f'  Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%')
    print(f'  Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%')

    # Показываем использование памяти GPU
    if torch.cuda.is_available():
        memory_allocated = torch.cuda.memory_allocated(0) / 1024**3
        memory_reserved = torch.cuda.memory_reserved(0) / 1024**3
        print(f'  GPU Memory - Allocated: {memory_allocated:.2f} GB, Reserved: {memory_reserved:.2f} GB')

print("\n" + "=" * 60)
print("ОБУЧЕНИЕ ЗАВЕРШЕНО")
print("=" * 60)

# Визуализация результатов
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

ax1.plot(range(1, epochs+1), train_losses, 'b-', label='Train Loss')
ax1.plot(range(1, epochs+1), val_losses, 'r-', label='Val Loss')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss')
ax1.set_title('Loss over epochs')
ax1.legend()
ax1.grid(True)

ax2.plot(range(1, epochs+1), train_accs, 'b-', label='Train Accuracy')
ax2.plot(range(1, epochs+1), val_accs, 'r-', label='Val Accuracy')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Accuracy (%)')
ax2.set_title('Accuracy over epochs')
ax2.legend()
ax2.grid(True)

plt.tight_layout()
plt.show()

# Тестирование на нескольких примерах
print("\nТестирование на нескольких примерах:")
model.eval()
with torch.no_grad():
    # Берем несколько примеров из тестового набора
    data, target = next(iter(test_loader))
    data, target = data[:8].to(device), target[:8].to(device)
    output = model(data)
    _, predicted = output.max(1)

    # Перемещаем обратно на CPU для отображения
    data = data.cpu()
    predicted = predicted.cpu()
    target = target.cpu()

    # Показываем изображения и предсказания
    fig, axes = plt.subplots(2, 4, figsize=(12, 6))
    for idx, ax in enumerate(axes.flat):
        if idx < 8:
            image = data[idx].squeeze().numpy()
            ax.imshow(image, cmap='gray')
            ax.set_title(f'Pred: {predicted[idx]}, True: {target[idx]}')
            ax.axis('off')
    plt.tight_layout()
    plt.show()

# Финальная статистика
print("\n" + "=" * 60)
print("ФИНАЛЬНАЯ СТАТИСТИКА")
print("=" * 60)
print(f"Финальная точность на валидации: {val_accs[-1]:.2f}%")
print(f"Финальная потеря на валидации: {val_losses[-1]:.4f}")

# Сохранение модели
torch.save({
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'train_losses': train_losses,
    'val_losses': val_losses,
    'train_accs': train_accs,
    'val_accs': val_accs,
}, 'mnist_cnn_model.pth')

print("Модель сохранена в 'mnist_cnn_model.pth'")

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import time
import numpy as np

# Установка устройства
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Используется устройство: {device}")

# Исправленная модель для CIFAR-10 (3 канала вместо 1)
class EnhancedCNN(nn.Module):
    def __init__(self):
        super(EnhancedCNN, self).__init__()
        # Изменяем первый слой на 3 канала (RGB)
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)  # Было 1, теперь 3
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(0.3)

        # Расчет размера после сверток для CIFAR-10 (32x32)
        # После conv1 + pool: 32x32 -> 16x16
        # После conv2 + pool: 16x16 -> 8x8
        # После conv3 + pool: 8x8 -> 4x4
        self.fc1 = nn.Linear(256 * 4 * 4, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 10)

        self.bn1 = nn.BatchNorm2d(64)
        self.bn2 = nn.BatchNorm2d(128)
        self.bn3 = nn.BatchNorm2d(256)

    def forward(self, x):
        x = self.pool(torch.relu(self.bn1(self.conv1(x))))
        x = self.pool(torch.relu(self.bn2(self.conv2(x))))
        x = self.pool(torch.relu(self.bn3(self.conv3(x))))

        x = x.view(-1, 256 * 4 * 4)
        x = self.dropout(x)
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Загрузка данных CIFAR-10
print("\nЗагрузка данных CIFAR-10...")
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),  # Аугментация
    transforms.RandomCrop(32, padding=4),  # Аугментация
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616))  # Стандартные значения для CIFAR-10
])

# Используем CIFAR-10
train_dataset = torchvision.datasets.CIFAR10(
    root='./data',
    train=True,
    download=True,
    transform=transform
)

test_dataset = torchvision.datasets.CIFAR10(
    root='./data',
    train=False,
    download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616))
    ])
)

# Увеличиваем batch size для вашей мощной GPU
batch_size = 256  # Можно увеличить до 512 если хватит памяти
print(f"Batch size: {batch_size}")

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)

# Классы CIFAR-10
classes = ('plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')

print(f"Обучающих примеров: {len(train_dataset)}")
print(f"Тестовых примеров: {len(test_dataset)}")
print(f"Размер изображений: {train_dataset[0][0].shape}")

# Создаем модель
model = EnhancedCNN().to(device)
print(f"\nМодель создана и перемещена на {device}")

# Функция для подсчета параметров
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

total_params = count_parameters(model)
print(f"Количество обучаемых параметров: {total_params:,}")

# Оптимизатор и функция потерь
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10)

# Функция обучения
def train_epoch(model, loader, criterion, optimizer, device, epoch):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for batch_idx, (data, target) in enumerate(loader):
        # Перемещаем данные на GPU
        data, target = data.to(device), target.to(device)

        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = output.max(1)
        total += target.size(0)
        correct += predicted.eq(target).sum().item()

        if batch_idx % 50 == 0:
            accuracy = 100. * correct / total
            print(f'Epoch: {epoch} | Batch: {batch_idx}/{len(loader)} | '
                  f'Loss: {loss.item():.4f} | Acc: {accuracy:.2f}%')

    accuracy = 100. * correct / total
    avg_loss = running_loss / len(loader)
    return avg_loss, accuracy

# Функция валидации
def validate(model, loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for data, target in loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            loss = criterion(output, target)

            running_loss += loss.item()
            _, predicted = output.max(1)
            total += target.size(0)
            correct += predicted.eq(target).sum().item()

    accuracy = 100. * correct / total
    avg_loss = running_loss / len(loader)
    return avg_loss, accuracy

# Обучение модели
print("\n" + "=" * 60)
print("НАЧАЛО ОБУЧЕНИЯ НА CIFAR-10")
print("=" * 60)

epochs = 15
train_losses, train_accs = [], []
val_losses, val_accs = [], []

for epoch in range(epochs):
    print(f'\n{"-" * 50}')
    print(f'Эпоха {epoch+1}/{epochs}')
    print(f'{"-" * 50}')

    # Обучение
    train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device, epoch+1)
    train_losses.append(train_loss)
    train_accs.append(train_acc)

    # Валидация
    val_loss, val_acc = validate(model, test_loader, criterion, device)
    val_losses.append(val_loss)
    val_accs.append(val_acc)

    # Обновление learning rate
    scheduler.step()

    print(f'\nИтоги эпохи {epoch+1}:')
    print(f'  Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%')
    print(f'  Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%')
    print(f'  Learning Rate: {optimizer.param_groups[0]["lr"]:.6f}')

    # Информация о памяти GPU
    if torch.cuda.is_available():
        memory_allocated = torch.cuda.memory_allocated(0) / 1024**3
        print(f'  GPU Memory: {memory_allocated:.2f}GB / 16.00GB')

print("\n" + "=" * 60)
print("ОБУЧЕНИЕ ЗАВЕРШЕНО")
print("=" * 60)

# Визуализация результатов
plt.figure(figsize=(15, 5))

# График потерь
plt.subplot(1, 2, 1)
plt.plot(range(1, epochs+1), train_losses, 'b-', linewidth=2, label='Train Loss')
plt.plot(range(1, epochs+1), val_losses, 'r-', linewidth=2, label='Val Loss')
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('Loss', fontsize=12)
plt.title('Loss over epochs', fontsize=14)
plt.legend()
plt.grid(True)

# График точности
plt.subplot(1, 2, 2)
plt.plot(range(1, epochs+1), train_accs, 'b-', linewidth=2, label='Train Accuracy')
plt.plot(range(1, epochs+1), val_accs, 'r-', linewidth=2, label='Val Accuracy')
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('Accuracy (%)', fontsize=12)
plt.title('Accuracy over epochs', fontsize=14)
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

# Тестирование на нескольких примерах
print("\nТестирование на примерах из тестового набора:")
model.eval()
with torch.no_grad():
    # Берем 10 примеров
    data, target = next(iter(test_loader))
    data, target = data[:10].to(device), target[:10].to(device)
    output = model(data)
    _, predicted = output.max(1)

    # Перемещаем обратно на CPU для отображения
    data = data.cpu()
    predicted = predicted.cpu()
    target = target.cpu()

    # Денормализация изображений
    mean = torch.tensor([0.4914, 0.4822, 0.4465]).view(3, 1, 1)
    std = torch.tensor([0.2470, 0.2435, 0.2616]).view(3, 1, 1)
    data_denorm = data * std + mean
    data_denorm = torch.clamp(data_denorm, 0, 1)

    # Показываем изображения и предсказания
    fig, axes = plt.subplots(2, 5, figsize=(15, 6))
    for idx, ax in enumerate(axes.flat):
        if idx < 10:
            image = data_denorm[idx].permute(1, 2, 0).numpy()
            ax.imshow(image)
            title = f'Pred: {classes[predicted[idx]]}\nTrue: {classes[target[idx]]}'
            ax.set_title(title, color='green' if predicted[idx] == target[idx] else 'red')
            ax.axis('off')

    plt.suptitle('Предсказания модели на тестовых данных CIFAR-10', fontsize=16)
    plt.tight_layout()
    plt.show()

# Матрица ошибок
print("\nМатрица ошибок (Confusion Matrix):")
model.eval()
all_preds = []
all_targets = []

with torch.no_grad():
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        output = model(data)
        _, preds = output.max(1)
        all_preds.extend(preds.cpu().numpy())
        all_targets.extend(target.cpu().numpy())

# Создаем матрицу ошибок
from sklearn.metrics import confusion_matrix
import seaborn as sns

cm = confusion_matrix(all_targets, all_preds)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=classes, yticklabels=classes)
plt.title('Confusion Matrix', fontsize=16)
plt.ylabel('True Label', fontsize=12)
plt.xlabel('Predicted Label', fontsize=12)
plt.tight_layout()
plt.show()

# Финальная статистика
print("\n" + "=" * 60)
print("ФИНАЛЬНАЯ СТАТИСТИКА")
print("=" * 60)
print(f"Финальная точность на валидации: {val_accs[-1]:.2f}%")
print(f"Финальная потеря на валидации: {val_losses[-1]:.4f}")
print(f"Лучшая точность на валидации: {max(val_accs):.2f}%")
print(f"Всего параметров модели: {total_params:,}")

# Сохранение модели
torch.save({
    'epoch': epochs,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'scheduler_state_dict': scheduler.state_dict(),
    'train_losses': train_losses,
    'val_losses': val_losses,
    'train_accs': train_accs,
    'val_accs': val_accs,
    'total_params': total_params,
}, 'cifar10_enhanced_cnn_corrected.pth')

print("\nМодель сохранена в 'cifar10_enhanced_cnn_corrected.pth'")

# Очистка памяти GPU
torch.cuda.empty_cache()
print("Память GPU очищена")

In [ ]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from tqdm import tqdm
import math
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import warnings
warnings.filterwarnings('ignore')


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

In [ ]:
df_id = pd.read_csv('selected_images/selected_images_info.csv')
df_id['person_id'].value_counts()

In [ ]:
df_id = df_id[['filename', 'person_id']]
print(df_id.head())
print(f"Всего изображений: {len(df_id)}")
print(f"Уникальных персон: {df_id['person_id'].nunique()}")

In [ ]:
img_dir = 'aligned_faces'

In [ ]:
# ==================== КОНФИГУРАЦИЯ ====================
class Config:
    def __init__(self):
        self.identity_df = df_id
        self.img_dir = img_dir
        self.max_classes = 350
        self.min_samples_per_person = 26
        self.seed = 42
        self.val_ratio = 0.15
        self.test_ratio = 0.15
        self.batch_size = 128
        self.num_workers = 0
        self.embedding_size = 512
        self.learning_rate = 0.001
        self.num_epochs = 25
        self.arcface_s = 32.0
        self.arcface_m = 0.5

# ==================== ОБРАБОТКА ДАННЫХ ====================
class CelebADataProcessor:
    def __init__(self, config):
        self.config = config
        self.identity_df = config.identity_df
        print(f"Всего данных: {len(self.identity_df)} изображений")
        print(f"Уникальных людей: {self.identity_df['person_id'].nunique()}")

    def filter_data(self):
        person_counts = self.identity_df['person_id'].value_counts()
        top_persons = person_counts.nlargest(self.config.max_classes).index
        self.filtered_df = self.identity_df[self.identity_df['person_id'].isin(top_persons)].copy()

        unique_ids = sorted(self.filtered_df['person_id'].unique())
        self.id_to_idx = {old_id: idx for idx, old_id in enumerate(unique_ids)}
        self.idx_to_id = {idx: old_id for old_id, idx in self.id_to_idx.items()}

        self.filtered_df['class_idx'] = self.filtered_df['person_id'].map(self.id_to_idx)

        print(f"\nПосле фильтрации:")
        print(f"  Всего изображений: {len(self.filtered_df)}")
        print(f"  Уникальных людей: {self.filtered_df['person_id'].nunique()}")
        print(f"  Диапазон меток: {self.filtered_df['class_idx'].min()} - {self.filtered_df['class_idx'].max()}")

        return len(unique_ids)

    def split_data_by_images(self):
        print(f"\nРазделение данных по изображениям (стратифицировано)...")

        train_df, temp_df = train_test_split(
            self.filtered_df,
            test_size=self.config.val_ratio + self.config.test_ratio,
            random_state=self.config.seed,
            stratify=self.filtered_df['class_idx']
        )

        val_df, test_df = train_test_split(
            temp_df,
            test_size=self.config.test_ratio/(self.config.val_ratio + self.config.test_ratio),
            random_state=self.config.seed,
            stratify=temp_df['class_idx']
        )

        self.train_df = train_df.reset_index(drop=True)
        self.val_df = val_df.reset_index(drop=True)
        self.test_df = test_df.reset_index(drop=True)

        print(f"Train: {len(self.train_df)} samples")
        print(f"Val: {len(self.val_df)} samples")
        print(f"Test: {len(self.test_df)} samples")

        return {
            'train': len(self.train_df),
            'val': len(self.val_df),
            'test': len(self.test_df),
            'num_persons': len(self.filtered_df['person_id'].unique())
        }

# ==================== ДАТАСЕТ ====================
class CelebAClassificationDataset(Dataset):
    def __init__(self, df, img_dir, transform=None):
        self.df = df.reset_index(drop=True)
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = os.path.join(self.img_dir, row['filename'])

        try:
            image = Image.open(img_path).convert('RGB')
        except:
            image = Image.new('RGB', (224, 224), color=(0, 0, 0))

        if self.transform:
            image = self.transform(image)

        label = int(row['class_idx'])
        return image, label

# ==================== МОДЕЛИ ====================
class SimpleFaceModel(nn.Module):
    """Простая и стабильная модель"""
    def __init__(self, num_classes=350, embedding_size=512):
        super().__init__()

        # Используем ResNet18
        self.backbone = models.resnet18(pretrained=True)
        in_features = self.backbone.fc.in_features
        self.backbone.fc = nn.Identity()

        # Простой embedding слой
        self.embedding = nn.Sequential(
            nn.Linear(in_features, embedding_size),
            nn.BatchNorm1d(embedding_size),
            nn.ReLU(inplace=True),
            nn.Dropout(0.3)
        )

        self.classifier = nn.Linear(embedding_size, num_classes)

        # Замораживаем только первые слои
        for name, param in self.backbone.named_parameters():
            if 'layer1' in name or 'conv1' in name or 'bn1' in name:
                param.requires_grad = False

        print(f"\nМодель параметров:")
        total_params = sum(p.numel() for p in self.parameters())
        trainable_params = sum(p.numel() for p in self.parameters() if p.requires_grad)
        print(f"  Всего: {total_params:,}")
        print(f"  Обучаемых: {trainable_params:,}")

    def forward(self, x, return_embedding=False):
        features = self.backbone(x)
        embedding = self.embedding(features)

        if return_embedding:
            return embedding

        logits = self.classifier(embedding)
        return logits, embedding

# ==================== ОСНОВНАЯ ФУНКЦИЯ ====================
def main():
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")

    config = Config()
    processor = CelebADataProcessor(config)

    num_classes = processor.filter_data()
    stats = processor.split_data_by_images()

    # Умеренные аугментации
    train_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomRotation(10),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                           std=[0.229, 0.224, 0.225])
    ])

    val_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                           std=[0.229, 0.224, 0.225])
    ])

    # Датасеты
    train_dataset = CelebAClassificationDataset(
        processor.train_df, config.img_dir, train_transform
    )
    val_dataset = CelebAClassificationDataset(
        processor.val_df, config.img_dir, val_transform
    )
    test_dataset = CelebAClassificationDataset(
        processor.test_df, config.img_dir, val_transform
    )

    # Даталоадеры
    train_loader = DataLoader(
        train_dataset, batch_size=config.batch_size,
        shuffle=True, num_workers=config.num_workers
    )
    val_loader = DataLoader(
        val_dataset, batch_size=config.batch_size,
        shuffle=False, num_workers=config.num_workers
    )
    test_loader = DataLoader(
        test_dataset, batch_size=config.batch_size,
        shuffle=False, num_workers=config.num_workers
    )

    # Создаем модель
    model = SimpleFaceModel(
        num_classes=num_classes,
        embedding_size=config.embedding_size
    ).to(device)

    # Простой оптимизатор
    optimizer = optim.Adam(
        model.parameters(),
        lr=config.learning_rate,
        weight_decay=1e-4
    )

    # StepLR scheduler
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

    criterion = nn.CrossEntropyLoss()

    os.makedirs('checkpoints', exist_ok=True)

    train_history = {
        'loss': [], 'acc': [],
        'val_loss': [], 'val_acc': [],
        'lr': []
    }

    best_val_acc = 0
    patience_counter = 0
    max_patience = 10

    print("\n" + "="*60)
    print("НАЧАЛО ОБУЧЕНИЯ")
    print("="*60)

    for epoch in range(config.num_epochs):
        print(f"\nEpoch {epoch+1}/{config.num_epochs}")
        print("-" * 40)

        # Обучение
        model.train()
        total_loss = 0
        correct = 0
        total = 0

        pbar = tqdm(train_loader, desc="Training")
        for batch_idx, (images, labels) in enumerate(pbar):
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            logits, _ = model(images, return_embedding=False)
            loss = criterion(logits, labels)
            loss.backward()

            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()

            total_loss += loss.item() * images.size(0)
            _, predicted = logits.max(1)
            correct += predicted.eq(labels).sum().item()
            total += images.size(0)

            current_acc = 100. * predicted.eq(labels).sum().item() / images.size(0)
            pbar.set_postfix({
                'loss': f'{loss.item():.4f}',
                'acc': f'{current_acc:.1f}%'
            })

        train_loss = total_loss / total
        train_acc = 100. * correct / total

        # Валидация
        model.eval()
        val_loss = 0
        val_correct = 0
        val_total = 0

        with torch.no_grad():
            for images, labels in tqdm(val_loader, desc="Validation"):
                images, labels = images.to(device), labels.to(device)
                logits, _ = model(images, return_embedding=False)
                loss = criterion(logits, labels)

                val_loss += loss.item() * images.size(0)
                _, predicted = logits.max(1)
                val_correct += predicted.eq(labels).sum().item()
                val_total += images.size(0)

        val_loss /= val_total
        val_acc = 100. * val_correct / val_total

        # Сохраняем историю
        train_history['loss'].append(train_loss)
        train_history['acc'].append(train_acc)
        train_history['val_loss'].append(val_loss)
        train_history['val_acc'].append(val_acc)
        train_history['lr'].append(optimizer.param_groups[0]['lr'])

        print(f"\nИтоги эпохи:")
        print(f"  Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%")
        print(f"  Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%")
        print(f"  Learning Rate: {optimizer.param_groups[0]['lr']:.2e}")

        # Сохраняем лучшую модель
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            patience_counter = 0
            checkpoint_path = f'checkpoints/best_model.pth'
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'train_loss': train_loss,
                'val_loss': val_loss,
                'train_acc': train_acc,
                'val_acc': val_acc,
            }, checkpoint_path)
            print(f"  ✓ Сохранена лучшая модель (Acc: {val_acc:.2f}%)")
        else:
            patience_counter += 1
            print(f"  Патience: {patience_counter}/{max_patience}")

            if patience_counter >= max_patience:
                print(f"  ⚠ Early stopping на эпохе {epoch+1}")
                break

        # Обновляем scheduler
        scheduler.step()

    # ==================== ВИЗУАЛИЗАЦИЯ ====================
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))

    axes[0, 0].plot(train_history['loss'], label='Train Loss', linewidth=2)
    axes[0, 0].plot(train_history['val_loss'], label='Val Loss', linewidth=2)
    axes[0, 0].set_xlabel('Epoch')
    axes[0, 0].set_ylabel('Loss')
    axes[0, 0].set_title('Loss History')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)

    axes[0, 1].plot(train_history['acc'], label='Train Acc', linewidth=2)
    axes[0, 1].plot(train_history['val_acc'], label='Val Acc', linewidth=2)
    axes[0, 1].set_xlabel('Epoch')
    axes[0, 1].set_ylabel('Accuracy (%)')
    axes[0, 1].set_title('Accuracy History')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)

    axes[1, 0].plot(train_history['lr'], linewidth=2)
    axes[1, 0].set_xlabel('Epoch')
    axes[1, 0].set_ylabel('Learning Rate')
    axes[1, 0].set_title('Learning Rate Schedule')
    axes[1, 0].grid(True, alpha=0.3)

    # Распределение меток
    axes[1, 1].hist(processor.train_df['class_idx'].values, bins=num_classes,
                    alpha=0.7, label='Train', density=True)
    axes[1, 1].hist(processor.val_df['class_idx'].values, bins=num_classes,
                    alpha=0.7, label='Val', density=True)
    axes[1, 1].set_xlabel('Class Index')
    axes[1, 1].set_ylabel('Density')
    axes[1, 1].set_title('Class Distribution')
    axes[1, 1].legend()
    axes[1, 1].grid(True, alpha=0.3)

    plt.tight_layout()
    plt.savefig('training_results_stable.png', dpi=150, bbox_inches='tight')
    plt.show()

    # ==================== ТЕСТИРОВАНИЕ ====================
    print("\n" + "="*60)
    print("ТЕСТИРОВАНИЕ ЛУЧШЕЙ МОДЕЛИ")
    print("="*60)

    # Загружаем лучшую модель
    checkpoint = torch.load('checkpoints/best_model.pth')
    checkpoint = torch.load('checkpoints/best_model.pth')
    model.load_state_dict(checkpoint['model_state_dict'])

    model.eval()
    test_loss = 0
    test_correct = 0
    test_total = 0

    with torch.no_grad():
        for images, labels in tqdm(test_loader, desc="Testing"):
            images, labels = images.to(device), labels.to(device)
            logits, _ = model(images, return_embedding=False)
            loss = criterion(logits, labels)

            test_loss += loss.item() * images.size(0)
            _, predicted = logits.max(1)
            test_correct += predicted.eq(labels).sum().item()
            test_total += images.size(0)

    test_loss /= test_total
    test_acc = 100. * test_correct / test_total

    print(f"\nРезультаты на тестовой выборке:")
    print(f"  Test Loss: {test_loss:.4f}")
    print(f"  Test Accuracy: {test_acc:.2f}%")
    print(f"  Correct/Total: {test_correct}/{test_total}")

    print("\n" + "="*60)
    print("ОБУЧЕНИЕ ЗАВЕРШЕНО!")
    print("="*60)
    print(f"Лучшая точность на валидации: {best_val_acc:.2f}%")
    print(f"Точность на тесте: {test_acc:.2f}%")

if __name__ == "__main__":
    main()

In [ ]:
# ==================== КОНФИГУРАЦИЯ ====================
class Config:
    def __init__(self):
        self.identity_df = df_id
        self.img_dir = img_dir
        self.max_classes = 350
        self.min_samples_per_person = 26
        self.seed = 42
        self.val_ratio = 0.15
        self.test_ratio = 0.15
        self.batch_size = 64
        self.num_workers = 0
        self.embedding_size = 512
        self.learning_rate = 0.0001
        self.num_epochs_ce = 25  # Эпох для CE
        self.num_epochs_arcface = 15  # Эпох для ArcFace fine-tuning
        self.arcface_s = 32.0  # Scale parameter
        self.arcface_m = 0.5   # Margin parameter
        self.arcface_easy_margin = True

# ==================== ОБРАБОТКА ДАННЫХ ====================
class CelebADataProcessor:
    def __init__(self, config):
        self.config = config
        self.identity_df = config.identity_df

        print(f"Всего данных: {len(self.identity_df)} изображений")
        print(f"Уникальных людей: {self.identity_df['person_id'].nunique()}")

    def filter_data(self):
        person_counts = self.identity_df['person_id'].value_counts()
        top_persons = person_counts.nlargest(self.config.max_classes).index
        self.filtered_df = self.identity_df[self.identity_df['person_id'].isin(top_persons)].copy()

        unique_ids = sorted(self.filtered_df['person_id'].unique())
        self.id_to_idx = {old_id: idx for idx, old_id in enumerate(unique_ids)}
        self.idx_to_id = {idx: old_id for old_id, idx in self.id_to_idx.items()}

        self.filtered_df['class_idx'] = self.filtered_df['person_id'].map(self.id_to_idx)

        print(f"\nПосле фильтрации:")
        print(f"  Всего изображений: {len(self.filtered_df)}")
        print(f"  Уникальных людей: {self.filtered_df['person_id'].nunique()}")
        print(f"  Диапазон меток: {self.filtered_df['class_idx'].min()} - {self.filtered_df['class_idx'].max()}")

        return len(unique_ids)

    def split_data_by_images(self):
        print(f"\nРазделение данных по изображениям (стратифицировано)...")

        train_df, temp_df = train_test_split(
            self.filtered_df,
            test_size=self.config.val_ratio + self.config.test_ratio,
            random_state=self.config.seed,
            stratify=self.filtered_df['class_idx']
        )

        val_df, test_df = train_test_split(
            temp_df,
            test_size=self.config.test_ratio/(self.config.val_ratio + self.config.test_ratio),
            random_state=self.config.seed,
            stratify=temp_df['class_idx']
        )

        self.train_df = train_df.reset_index(drop=True)
        self.val_df = val_df.reset_index(drop=True)
        self.test_df = test_df.reset_index(drop=True)

        print(f"Train: {len(self.train_df)} samples")
        print(f"Val: {len(self.val_df)} samples")
        print(f"Test: {len(self.test_df)} samples")

        return {
            'train': len(self.train_df),
            'val': len(self.val_df),
            'test': len(self.test_df),
            'num_persons': len(self.filtered_df['person_id'].unique())
        }

# ==================== ДАТАСЕТ ====================
class CelebAClassificationDataset(Dataset):
    def __init__(self, df, img_dir, transform=None):
        self.df = df.reset_index(drop=True)
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = os.path.join(self.img_dir, row['filename'])

        try:
            image = Image.open(img_path).convert('RGB')
        except:
            image = Image.new('RGB', (224, 224), color=(0, 0, 0))

        if self.transform:
            image = self.transform(image)

        label = int(row['class_idx'])
        return image, label

# ==================== МОДЕЛИ ====================
class SimpleFaceModel(nn.Module):
    def __init__(self, num_classes=300, embedding_size=512):
        super().__init__()

        self.backbone = models.resnet18(pretrained=True)
        in_features = self.backbone.fc.in_features

        # Заменяем последний слой
        self.backbone.fc = nn.Identity()

        # Эмбеддинг слой (для совместимости с ArcFace)
        self.embedding = nn.Sequential(
            nn.Linear(in_features, embedding_size),
            nn.BatchNorm1d(embedding_size),
            nn.ReLU(inplace=True),
            nn.Dropout(0.3)
        )

        # Классификатор
        self.classifier = nn.Linear(embedding_size, num_classes)

        # Замораживаем первые слои
        for name, param in self.backbone.named_parameters():
            if 'layer1' in name or 'conv1' in name or 'bn1' in name:
                param.requires_grad = False

    def forward(self, x, labels=None):
        features = self.backbone(x)
        embedding = self.embedding(features)
        logits = self.classifier(embedding)

        if labels is not None:
            return logits, embedding
        return logits, embedding


class ArcMarginProduct(nn.Module):
    """ArcFace loss слой"""
    def __init__(self, in_features, out_features, s=32.0, m=0.3, easy_margin=True):
        super().__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.s = s
        self.m = m

        # Easy margin для стабильности
        self.easy_margin = easy_margin

        self.weight = nn.Parameter(torch.FloatTensor(out_features, in_features))
        nn.init.xavier_normal_(self.weight)

        # Инициализация весов
        nn.init.xavier_normal_(self.weight)

        # Предварительные вычисления
        self.cos_m = math.cos(m)
        self.sin_m = math.sin(m)

        # Угол для easy margin
        self.th = math.cos(math.pi - m)
        self.mm = math.sin(math.pi - m) * m

        print(f"ArcFace инициализирован: s={s}, m={m}")

    def forward(self, input, label):
        # Нормализуем входные эмбеддинги
        cosine = F.linear(F.normalize(input, p=2, dim=1),
                         F.normalize(self.weight, p=2, dim=1))

        # Обеспечиваем численную стабильность
        cosine = cosine.clamp(-1 + 1e-7, 1 - 1e-7)

        # Вычисляем синус
        sine = torch.sqrt(1.0 - torch.pow(cosine, 2) + 1e-7)

        # Косинус угла с маржой
        phi = cosine * self.cos_m - sine * self.sin_m

        if self.easy_margin:
            phi = torch.where(cosine > 0, phi, cosine)
        else:
            phi = torch.where(cosine > self.th, phi, cosine - self.mm)

        # One-hot encoding меток
        one_hot = torch.zeros_like(cosine)
        one_hot.scatter_(1, label.view(-1, 1).long(), 1)

        # Комбинируем
        output = (one_hot * phi) + ((1.0 - one_hot) * cosine)
        output *= self.s

        return output

class ArcFaceModel(nn.Module):
    def __init__(self, num_classes=300, embedding_size=512, s=8.0, m=0.5):
        super().__init__()

        # Backbone (ResNet18)
        self.backbone = models.resnet18(pretrained=True)
        in_features = self.backbone.fc.in_features
        self.backbone.fc = nn.Identity()

        # Embedding слой С BatchNorm
        self.embedding = nn.Sequential(
            nn.Linear(in_features, embedding_size),
            nn.BatchNorm1d(embedding_size),
            nn.ReLU(inplace=True),
            nn.Dropout(0.3)
        )

        # ArcFace слой
        self.arcface = ArcMarginProduct(
            embedding_size,
            num_classes,
            s=s,
            m=m,
            easy_margin=True
        )

        print(f"\nArcFace Model создана:")
        print(f"  Количество классов: {num_classes}")
        print(f"  Embedding size: {embedding_size}")
        print(f"  Scale s: {s}")
        print(f"  Margin m: {m}")

    def load_from_ce_model(self, ce_model_path, device='cpu'):
        """Загрузка весов из CE модели"""
        print(f"\n{'='*50}")
        print("ЗАГРУЗКА ВЕСОВ ИЗ CE МОДЕЛИ")
        print(f"{'='*50}")

        try:
            # Загружаем CE модель
            if os.path.exists(ce_model_path):
                ce_checkpoint = torch.load(ce_model_path, map_location=device)

                # Проверяем формат чекпоинта
                if isinstance(ce_checkpoint, dict) and 'model_state_dict' in ce_checkpoint:
                    ce_state_dict = ce_checkpoint['model_state_dict']
                else:
                    ce_state_dict = ce_checkpoint

                # Загружаем веса в ArcFace модель
                self_state_dict = self.state_dict()

                # 1. Загружаем backbone (полное совпадение имен)
                for name, param in self.backbone.named_parameters():
                    ce_key = f'backbone.{name}'
                    if ce_key in ce_state_dict:
                        param.data.copy_(ce_state_dict[ce_key])

                # 2. Загружаем embedding слой (полное совпадение имен)
                for name, param in self.embedding.named_parameters():
                    ce_key = f'embedding.{name}'
                    if ce_key in ce_state_dict:
                        param.data.copy_(ce_state_dict[ce_key])

                # 3. Загружаем BatchNorm статистику
                for name, buffer in self.embedding.named_buffers():
                    ce_key = f'embedding.{name}'
                    if ce_key in ce_state_dict:
                        buffer.copy_(ce_state_dict[ce_key])

                # 4. Инициализируем ArcFace слой из весов классификатора CE
                if 'classifier.weight' in ce_state_dict:
                    ce_weight = ce_state_dict['classifier.weight']

                    # Нормализуем веса классификатора
                    ce_weight_norm = F.normalize(ce_weight, p=2, dim=1)
                    ce_weight_norm = ce_weight_norm * 0.1

                    # Копируем в ArcFace слой
                    self.arcface.weight.data.copy_(ce_weight_norm)

                    print(f"✓ Backbone загружен")
                    print(f"✓ Embedding слой загружен")
                    print(f"✓ ArcFace инициализирован из classifier.weight")
                    print(f"✓ Scale s = {self.arcface.s}")

                    # Проверка размерностей
                    print(f"\nПроверка размерностей:")
                    print(f"  classifier.weight: {ce_weight.shape}")
                    print(f"  arcface.weight: {self.arcface.weight.shape}")
                    print(f"  Нормализованные веса: {ce_weight_norm.norm(dim=1).mean():.4f}")

                else:
                    print("⚠ classifier.weight не найден, инициализируем ArcFace случайно")
                    self.arcface.s = 8.0  # Меньший scale

            else:
                print(f"⚠ Файл {ce_model_path} не найден")

        except Exception as e:
            print(f"❌ Ошибка при загрузке: {e}")
            print("Обучаем ArcFace с нуля...")
            self.arcface.s = 16.0  # Меньший scale при обучении с нуля

    def forward(self, x, labels=None):
        # Извлекаем фичи
        features = self.backbone(x)

        # Получаем эмбеддинги
        embeddings = self.embedding(features)

        # Обязательно нормализуем для ArcFace
        embeddings = F.normalize(embeddings, p=2, dim=1)

        if labels is not None:
            # Для обучения: возвращаем logits и эмбеддинги
            logits = self.arcface(embeddings, labels)
            return logits, embeddings

        # Для инференса: возвращаем только эмбеддинги
        return embeddings

# ==================== ФУНКЦИИ ОБУЧЕНИЯ ====================
def train_epoch_ce(model, loader, optimizer, criterion, device):
    """Обучение CE модели для одной эпохи"""
    model.train()
    total_loss = 0
    correct = 0
    total = 0

    pbar = tqdm(loader, desc="Training CE")
    for images, labels in pbar:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        logits, _ = model(images)
        loss = criterion(logits, labels)
        loss.backward()

        # Gradient clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

        optimizer.step()

        total_loss += loss.item() * images.size(0)
        _, predicted = logits.max(1)
        correct += predicted.eq(labels).sum().item()
        total += images.size(0)

        current_acc = 100. * predicted.eq(labels).sum().item() / images.size(0)
        pbar.set_postfix({
            'loss': f'{loss.item():.4f}',
            'acc': f'{current_acc:.1f}%'
        })

    avg_loss = total_loss / total if total > 0 else 0
    accuracy = 100. * correct / total if total > 0 else 0

    return avg_loss, accuracy


def train_epoch_arcface(model, loader, optimizer, criterion, device, epoch):
    """Обучение ArcFace модели для одной эпохи"""
    model.train()
    total_loss = 0
    correct = 0
    total = 0

    # Progressive scaling
    if epoch < 3:
        model.arcface.s = 8.0
    elif epoch < 6:
        model.arcface.s = 16.0
    elif epoch < 8:
        model.arcface.s = 24.0
    else:
        model.arcface.s = 32.0

    pbar = tqdm(loader, desc=f"Training ArcFace (s={model.arcface.s:.1f})")
    for batch_idx, (images, labels) in enumerate(pbar):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        logits, _ = model(images, labels)

        if batch_idx == 0 and epoch == 0:
            print(f"\n[DEBUG] Первый батч эпохи {epoch+1}:")
            print(f"  Logits min/max: {logits.min():.4f}/{logits.max():.4f}")
            print(f"  Logits mean/std: {logits.mean():.4f}/{logits.std():.4f}")
            print(f"  Scale s: {model.arcface.s}")

        loss = criterion(logits, labels)
        loss.backward()

        # Gradient clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

        optimizer.step()

        # Вычисляем accuracy (для ArcFace нужен softmax)
        with torch.no_grad():
            probs = F.softmax(logits, dim=1)
            predicted = probs.argmax(dim=1)

            total_loss += loss.item() * images.size(0)
            correct += predicted.eq(labels).sum().item()
            total += images.size(0)

            current_acc = 100. * predicted.eq(labels).sum().item() / images.size(0)
            pbar.set_postfix({
                'loss': f'{loss.item():.4f}',
                'acc': f'{current_acc:.1f}%'
            })

    avg_loss = total_loss / total if total > 0 else 0
    accuracy = 100. * correct / total if total > 0 else 0

    return avg_loss, accuracy


def validate(model, loader, criterion, device, model_type='ce'):
    """Валидация модели"""
    model.eval()
    val_loss = 0
    val_correct = 0
    val_total = 0

    with torch.no_grad():
        for images, labels in tqdm(loader, desc="Validation"):
            images, labels = images.to(device), labels.to(device)

            if model_type == 'ce':
                logits, _ = model(images)
                _, predicted = logits.max(1)
            else:  # arcface
                logits, _ = model(images, labels)
                probs = F.softmax(logits, dim=1)
                predicted = probs.argmax(dim=1)

            loss = criterion(logits, labels)
            val_loss += loss.item() * images.size(0)
            val_correct += predicted.eq(labels).sum().item()
            val_total += images.size(0)

    avg_loss = val_loss / val_total
    accuracy = 100. * val_correct / val_total

    return avg_loss, accuracy

def main():
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")

    config = Config()

    # ==================== 1. ПОДГОТОВКА ДАННЫХ ====================
    print("\n" + "="*60)
    print("ПОДГОТОВКА ДАННЫХ")
    print("="*60)

    # Копируем ваш работающий код подготовки данных
    processor = CelebADataProcessor(config)
    num_classes = processor.filter_data()
    stats = processor.split_data_by_images()

    # Трансформации (такие же как в работающем коде)
    train_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomRotation(10),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                           std=[0.229, 0.224, 0.225])
    ])

    val_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                           std=[0.229, 0.224, 0.225])
    ])

    # Датасеты
    train_dataset = CelebAClassificationDataset(
        processor.train_df, config.img_dir, train_transform
    )
    val_dataset = CelebAClassificationDataset(
        processor.val_df, config.img_dir, val_transform
    )
    test_dataset = CelebAClassificationDataset(
        processor.test_df, config.img_dir, val_transform
    )

    # Даталоадеры
    train_loader = DataLoader(
        train_dataset, batch_size=config.batch_size,
        shuffle=True, num_workers=config.num_workers,
        pin_memory=True
    )
    val_loader = DataLoader(
        val_dataset, batch_size=config.batch_size,
        shuffle=False, num_workers=config.num_workers
    )
    test_loader = DataLoader(
        test_dataset, batch_size=config.batch_size,
        shuffle=False, num_workers=config.num_workers
    )

    # Создаем папку для чекпоинтов
    os.makedirs('checkpoints', exist_ok=True)

    # ==================== 2. ЗАГРУЗКА ОБУЧЕННОЙ CE МОДЕЛИ ====================
    print("\n" + "="*60)
    print("ЗАГРУЗКА ОБУЧЕННОЙ CE МОДЕЛИ")
    print("="*60)

    # Проверяем, есть ли уже обученная CE модель
    ce_checkpoint_path = 'checkpoints/best_model_ce.pth'

    if not os.path.exists(ce_checkpoint_path):
        print(f"Файл {ce_checkpoint_path} не найден")
        print("Обучаем CE модель сначала...")

        # Обучаем CE модель
        model_ce = SimpleFaceModel(
            num_classes=num_classes,
            embedding_size=config.embedding_size
        ).to(device)

        optimizer_ce = optim.Adam(
            model_ce.parameters(),
            lr=0.001,
            weight_decay=1e-4
        )

        scheduler_ce = optim.lr_scheduler.StepLR(optimizer_ce, step_size=10, gamma=0.5)
        criterion_ce = nn.CrossEntropyLoss()

        best_ce_acc = 0
        for epoch in range(config.num_epochs_ce):
            print(f"\nEpoch {epoch+1}/{config.num_epochs_ce} (CE)")
            print("-" * 40)

            train_loss, train_acc = train_epoch_ce(
                model_ce, train_loader, optimizer_ce, criterion_ce, device
            )

            val_loss, val_acc = validate(
                model_ce, val_loader, criterion_ce, device, 'ce'
            )

            print(f"\nИтоги эпохи:")
            print(f"  Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%")
            print(f"  Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%")

            if val_acc > best_ce_acc:
                best_ce_acc = val_acc
                torch.save({
                    'epoch': epoch,
                    'model_state_dict': model_ce.state_dict(),
                    'val_acc': val_acc,
                    'optimizer_state_dict': optimizer_ce.state_dict(),
                }, ce_checkpoint_path)
                print(f"  ✓ Сохранена CE модель (Acc: {val_acc:.2f}%)")

            scheduler_ce.step()

    # ==================== 3. FINE-TUNING С ARCFACE ====================
    print("\n" + "="*60)
    print("FINE-TUNING С ARCFACE LOSS")
    print("="*60)

    # Создаем ArcFace модель
    model_arcface = ArcFaceModel(
        num_classes=num_classes,
        embedding_size=config.embedding_size,
        s=8.0,  # Начинаем с меньшего scale
        m=config.arcface_m
    ).to(device)

    # Загружаем веса из CE модели
    model_arcface.load_from_ce_model(ce_checkpoint_path, device)

    # Проверка после загрузки
    print("\n" + "="*50)
    print("ПРОВЕРКА ПОСЛЕ ЗАГРУЗКИ ВЕСОВ")
    print("="*50)

    model_arcface.eval()
    with torch.no_grad():
        test_images, test_labels = next(iter(train_loader))
        test_images = test_images[:4].to(device)
        test_labels = test_labels[:4].to(device)

        # Проверка forward pass
        logits, embeddings = model_arcface(test_images, test_labels)

        print(f"\n1. Проверка размерностей:")
        print(f"   Logits shape: {logits.shape}")
        print(f"   Embeddings shape: {embeddings.shape}")

        print(f"\n2. Проверка численных значений:")
        print(f"   Logits range: [{logits.min():.4f}, {logits.max():.4f}]")
        print(f"   Logits mean/std: {logits.mean():.4f} / {logits.std():.4f}")

        print(f"\n3. Проверка accuracy:")
        probs = F.softmax(logits, dim=1)
        predicted = probs.argmax(dim=1)
        accuracy = (predicted == test_labels).float().mean().item()
        print(f"   Accuracy на тестовом батче: {accuracy:.2%}")

        # Проверка потерь
        test_loss = nn.CrossEntropyLoss()(logits, test_labels)
        print(f"   Loss: {test_loss.item():.4f}")

    # Оптимизатор для fine-tuning
    optimizer_arc = optim.AdamW(
        model_arcface.parameters(),
        lr=config.learning_rate,  # Малый LR для fine-tuning
        weight_decay=0.0005
    )

    # Scheduler
    scheduler_arc = optim.lr_scheduler.CosineAnnealingLR(
        optimizer_arc,
        T_max=config.num_epochs_arcface,
        eta_min=1e-6
    )

    # Loss function с label smoothing
    criterion_arc = nn.CrossEntropyLoss(label_smoothing=0.1)

    # Обучение ArcFace
    print("\n" + "="*60)
    print("НАЧАЛО ОБУЧЕНИЯ ARCFACE")
    print("="*60)

    best_arc_acc = 0
    arc_history = {'loss': [], 'acc': [], 'val_loss': [], 'val_acc': []}

    for epoch in range(config.num_epochs_arcface):
        print(f"\n{'='*50}")
        print(f"Epoch {epoch+1}/{config.num_epochs_arcface} - ArcFace Fine-tuning")
        print(f"{'='*50}")

        # Обучение
        train_loss, train_acc = train_epoch_arcface(
            model_arcface, train_loader, optimizer_arc, criterion_arc, device, epoch
        )

        # Валидация
        val_loss, val_acc = validate(
            model_arcface, val_loader, criterion_arc, device, 'arcface'
        )

        # Сохраняем историю
        arc_history['loss'].append(train_loss)
        arc_history['acc'].append(train_acc)
        arc_history['val_loss'].append(val_loss)
        arc_history['val_acc'].append(val_acc)

        print(f"\n📊 Итоги эпохи:")
        print(f"  Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%")
        print(f"  Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%")
        print(f"  Scale s: {model_arcface.arcface.s}")
        print(f"  Learning Rate: {optimizer_arc.param_groups[0]['lr']:.2e}")

        # Сохраняем лучшую модель
        if val_acc > best_arc_acc:
            best_arc_acc = val_acc
            torch.save({
                'epoch': epoch,
                'model_state_dict': model_arcface.state_dict(),
                'optimizer_state_dict': optimizer_arc.state_dict(),
                'val_acc': val_acc,
                'scale': model_arcface.arcface.s,
            }, 'checkpoints/best_model_arcface.pth')
            print(f"  💾 Сохранена лучшая ArcFace модель (Val Acc: {val_acc:.2f}%)")

        # Обновляем scheduler
        scheduler_arc.step()

    # ==================== 4. ТЕСТИРОВАНИЕ ====================
    print("\n" + "="*60)
    print("ТЕСТИРОВАНИЕ ARCFACE МОДЕЛИ")
    print("="*60)

    # Загружаем лучшую ArcFace модель
    arc_checkpoint = torch.load('checkpoints/best_model_arcface.pth')
    model_arcface.load_state_dict(arc_checkpoint['model_state_dict'])

    # Тестирование
    test_loss, test_acc = validate(
        model_arcface, test_loader, criterion_arc, device, 'arcface'
    )

    print(f"\n📊 Результаты ArcFace на тестовой выборке:")
    print(f"  Test Loss: {test_loss:.4f}")
    print(f"  Test Accuracy: {test_acc:.2f}%")
    print(f"  Final Scale s: {model_arcface.arcface.s}")

    # ==================== 5. ВИЗУАЛИЗАЦИЯ ====================
    print("\n" + "="*60)
    print("ВИЗУАЛИЗАЦИЯ РЕЗУЛЬТАТОВ")
    print("="*60)

    # Создаем графики
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))

    # Loss history
    axes[0, 0].plot(arc_history['loss'], label='Train Loss', linewidth=2)
    axes[0, 0].plot(arc_history['val_loss'], label='Val Loss', linewidth=2)
    axes[0, 0].set_xlabel('Epoch')
    axes[0, 0].set_ylabel('Loss')
    axes[0, 0].set_title('ArcFace: Loss History')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)

    # Accuracy history
    axes[0, 1].plot(arc_history['acc'], label='Train Acc', linewidth=2)
    axes[0, 1].plot(arc_history['val_acc'], label='Val Acc', linewidth=2)
    axes[0, 1].set_xlabel('Epoch')
    axes[0, 1].set_ylabel('Accuracy (%)')
    axes[0, 1].set_title('ArcFace: Accuracy History')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)

    # Learning rate history
    lr_history = [group['lr'] for group in optimizer_arc.param_groups]
    axes[1, 0].plot(lr_history, linewidth=2)
    axes[1, 0].set_xlabel('Epoch')
    axes[1, 0].set_ylabel('Learning Rate')
    axes[1, 0].set_title('Learning Rate Schedule')
    axes[1, 0].grid(True, alpha=0.3)

    # Scale s history
    scale_progression = [32, 32, 32, 48, 48, 48, 64, 64, 64, 64][:len(arc_history['loss'])]
    axes[1, 1].plot(scale_progression, linewidth=2, color='green')
    axes[1, 1].set_xlabel('Epoch')
    axes[1, 1].set_ylabel('Scale s')
    axes[1, 1].set_title('ArcFace Scale Progression')
    axes[1, 1].grid(True, alpha=0.3)

    plt.tight_layout()
    plt.savefig('arcface_training_results.png', dpi=150, bbox_inches='tight')
    plt.show()

    print("\n" + "="*60)
    print("ОБУЧЕНИЕ ARCFACE ЗАВЕРШЕНО!")
    print("="*60)
    print(f"Лучшая точность на валидации: {best_arc_acc:.2f}%")
    print(f"Точность на тесте: {test_acc:.2f}%")


if __name__ == "__main__":
    main()

ЕЩЕ ОДИН ВАРИАНТ

In [ ]:
import os
import math
import logging
from datetime import datetime
import numpy as np
from PIL import Image
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

# ==================== КОНФИГУРАЦИЯ ====================
class Config:
    def __init__(self):
        # Эти переменные должны быть определены в вашем окружении
        # self.identity_df = df_id
        # self.img_dir = img_dir
        self.max_classes = 350
        self.min_samples_per_person = 26
        self.seed = 42
        self.val_ratio = 0.15
        self.test_ratio = 0.15
        self.batch_size = 64
        self.num_workers = 0
        self.embedding_size = 512
        self.learning_rate = 0.0001
        self.num_epochs_ce = 15
        self.num_epochs_arcface = 10
        self.arcface_s = 32.0
        self.arcface_m = 0.5
        self.arcface_easy_margin = True

# ==================== ОБРАБОТКА ДАННЫХ ====================
class CelebADataProcessor:
    def __init__(self, config):
        self.config = config
        self.identity_df = config.identity_df

        print(f"Всего данных: {len(self.identity_df)} изображений")
        print(f"Уникальных людей: {self.identity_df['person_id'].nunique()}")

    def filter_data(self):
        person_counts = self.identity_df['person_id'].value_counts()
        top_persons = person_counts.nlargest(self.config.max_classes).index
        self.filtered_df = self.identity_df[self.identity_df['person_id'].isin(top_persons)].copy()

        unique_ids = sorted(self.filtered_df['person_id'].unique())
        self.id_to_idx = {old_id: idx for idx, old_id in enumerate(unique_ids)}
        self.idx_to_id = {idx: old_id for old_id, idx in self.id_to_idx.items()}

        self.filtered_df['class_idx'] = self.filtered_df['person_id'].map(self.id_to_idx)

        print(f"\nПосле фильтрации:")
        print(f"  Всего изображений: {len(self.filtered_df)}")
        print(f"  Уникальных людей: {self.filtered_df['person_id'].nunique()}")
        print(f"  Диапазон меток: {self.filtered_df['class_idx'].min()} - {self.filtered_df['class_idx'].max()}")

        return len(unique_ids)

    def split_data_by_images(self):
        print(f"\nРазделение данных по изображениям (стратифицировано)...")

        train_df, temp_df = train_test_split(
            self.filtered_df,
            test_size=self.config.val_ratio + self.config.test_ratio,
            random_state=self.config.seed,
            stratify=self.filtered_df['class_idx']
        )

        val_df, test_df = train_test_split(
            temp_df,
            test_size=self.config.test_ratio/(self.config.val_ratio + self.config.test_ratio),
            random_state=self.config.seed,
            stratify=temp_df['class_idx']
        )

        self.train_df = train_df.reset_index(drop=True)
        self.val_df = val_df.reset_index(drop=True)
        self.test_df = test_df.reset_index(drop=True)

        print(f"Train: {len(self.train_df)} samples")
        print(f"Val: {len(self.val_df)} samples")
        print(f"Test: {len(self.test_df)} samples")

        return {
            'train': len(self.train_df),
            'val': len(self.val_df),
            'test': len(self.test_df),
            'num_persons': len(self.filtered_df['person_id'].unique())
        }

# ==================== ДАТАСЕТ ====================
class CelebAClassificationDataset(Dataset):
    def __init__(self, df, img_dir, transform=None):
        self.df = df.reset_index(drop=True)
        self.img_dir = img_dir
        self.transform = transform

        # Настройка логирования
        logging.basicConfig(
            level=logging.WARNING,
            format='%(asctime)s - %(levelname)s - %(message)s'
        )
        self.logger = logging.getLogger(__name__)

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = os.path.join(self.img_dir, row['filename'])

        try:
            image = Image.open(img_path).convert('RGB')

            # Проверка валидности изображения
            if image.mode != 'RGB':
                image = image.convert('RGB')

            # Проверка размера
            if image.size[0] < 10 or image.size[1] < 10:
                self.logger.warning(f"Image {img_path} too small: {image.size}")
                image = image.resize((224, 224), Image.BILINEAR)

        except Exception as e:
            self.logger.warning(f"Cannot load image {img_path}: {e}")
            # Создаем черное изображение
            image = Image.new('RGB', (224, 224), color=(0, 0, 0))

        if self.transform:
            image = self.transform(image)

        label = int(row['class_idx'])
        return image, label

# ==================== МОДЕЛИ ====================
class SimpleFaceModel(nn.Module):
    def __init__(self, num_classes=350, embedding_size=512):
        super().__init__()

        self.backbone = models.resnet18(pretrained=True)
        in_features = self.backbone.fc.in_features

        # Заменяем последний слой
        self.backbone.fc = nn.Identity()

        # Эмбеддинг слой
        self.embedding = nn.Sequential(
            nn.Linear(in_features, embedding_size),
            nn.BatchNorm1d(embedding_size),
            nn.ReLU(inplace=True),
            nn.Dropout(0.3)
        )

        # Классификатор
        self.classifier = nn.Linear(embedding_size, num_classes)

        # Замораживаем первые слои
        for name, param in self.backbone.named_parameters():
            if 'layer1' in name or 'conv1' in name or 'bn1' in name:
                param.requires_grad = False

    def forward(self, x):
        features = self.backbone(x)
        embedding = self.embedding(features)
        logits = self.classifier(embedding)
        return logits, embedding


class ArcMarginProduct(nn.Module):
    """ArcFace loss слой"""
    def __init__(self, in_features, out_features, s=32.0, m=0.5, easy_margin=True):
        super().__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.s = s
        self.m = m
        self.easy_margin = easy_margin

        self.weight = nn.Parameter(torch.FloatTensor(out_features, in_features))
        nn.init.xavier_uniform_(self.weight)

        # Предварительные вычисления
        self.cos_m = math.cos(m)
        self.sin_m = math.sin(m)

        # Угол для easy margin
        self.th = math.cos(math.pi - m)
        self.mm = math.sin(math.pi - m) * m

        print(f"ArcFace инициализирован: s={s}, m={m}")

    def forward(self, input, label):
        # Нормализуем входные эмбеддинги
        cosine = F.linear(F.normalize(input, p=2, dim=1),
                         F.normalize(self.weight, p=2, dim=1))

        # Обеспечиваем численную стабильность
        cosine = cosine.clamp(-1 + 1e-7, 1 - 1e-7)

        # Вычисляем синус
        sine = torch.sqrt(1.0 - torch.pow(cosine, 2) + 1e-7)

        # Косинус угла с маржой
        phi = cosine * self.cos_m - sine * self.sin_m

        if self.easy_margin:
            phi = torch.where(cosine > 0, phi, cosine)
        else:
            phi = torch.where(cosine > self.th, phi, cosine - self.mm)

        # One-hot encoding меток
        one_hot = torch.zeros_like(cosine)
        one_hot.scatter_(1, label.view(-1, 1).long(), 1)

        # Комбинируем
        output = (one_hot * phi) + ((1.0 - one_hot) * cosine)
        output *= self.s

        return output

class ArcFaceModel(nn.Module):
    def __init__(self, num_classes=350, embedding_size=512, s=8.0, m=0.5):
        super().__init__()

        # Backbone (ResNet18)
        self.backbone = models.resnet18(pretrained=True)
        in_features = self.backbone.fc.in_features
        self.backbone.fc = nn.Identity()

        # Embedding слой
        self.embedding = nn.Sequential(
            nn.Linear(in_features, embedding_size),
            nn.BatchNorm1d(embedding_size),
            nn.ReLU(inplace=True),
            nn.Dropout(0.3)
        )

        # ArcFace слой
        self.arcface = ArcMarginProduct(
            embedding_size,
            num_classes,
            s=s,
            m=m,
            easy_margin=True
        )

        print(f"\nArcFace Model создана:")
        print(f"  Количество классов: {num_classes}")
        print(f"  Embedding size: {embedding_size}")
        print(f"  Scale s: {s}")
        print(f"  Margin m: {m}")

    def load_from_ce_model(self, ce_model_path, device='cpu'):
        """Загрузка весов из CE модели"""
        print(f"\n{'='*50}")
        print("ЗАГРУЗКА ВЕСОВ ИЗ CE МОДЕЛИ")
        print(f"{'='*50}")

        try:
            if not os.path.exists(ce_model_path):
                print(f"⚠ Файл {ce_model_path} не найден")
                print("Обучаем ArcFace с нуля...")
                self._initialize_random()
                return False

            # Загружаем CE модель
            ce_checkpoint = torch.load(ce_model_path, map_location=device)

            # Проверяем формат чекпоинта
            if isinstance(ce_checkpoint, dict) and 'model_state_dict' in ce_checkpoint:
                ce_state_dict = ce_checkpoint['model_state_dict']
            else:
                ce_state_dict = ce_checkpoint

            # 1. Загружаем backbone
            backbone_dict = {}
            for key, value in ce_state_dict.items():
                if key.startswith('backbone.'):
                    # Убираем 'backbone.' из ключа
                    new_key = key.replace('backbone.', '', 1)
                    backbone_dict[new_key] = value

            if backbone_dict:
                missing_keys, unexpected_keys = self.backbone.load_state_dict(backbone_dict, strict=False)
                print("✓ Backbone загружен")
                if missing_keys:
                    print(f"  Отсутствующие ключи: {missing_keys[:5]}{'...' if len(missing_keys) > 5 else ''}")
                if unexpected_keys:
                    print(f"  Неожиданные ключи: {unexpected_keys[:5]}{'...' if len(unexpected_keys) > 5 else ''}")
            else:
                print("⚠ Backbone веса не найдены")

            # 2. Загружаем embedding слой
            embedding_dict = {}
            for key, value in ce_state_dict.items():
                if key.startswith('embedding.'):
                    new_key = key.replace('embedding.', '', 1)
                    embedding_dict[new_key] = value

            if embedding_dict:
                self.embedding.load_state_dict(embedding_dict, strict=False)
                print("✓ Embedding слой загружен")
            else:
                print("⚠ Embedding веса не найдены")

            # 3. Инициализируем ArcFace слой
            self._initialize_arcface()
            print("✓ ArcFace инициализирован случайно")
            print(f"✓ Scale s = {self.arcface.s}")

            return True

        except Exception as e:
            print(f"❌ Ошибка при загрузке: {e}")
            print("Обучаем ArcFace с нуля...")
            self._initialize_random()
            return False

    def _initialize_arcface(self):
        """Инициализация ArcFace слоя"""
        nn.init.xavier_uniform_(self.arcface.weight)
        self.arcface.s = 8.0

    def _initialize_random(self):
        """Полная случайная инициализация"""
        for module in [self.embedding, self.arcface]:
            for m in module.modules():
                if isinstance(m, nn.Linear):
                    nn.init.xavier_uniform_(m.weight)
                    if m.bias is not None:
                        nn.init.zeros_(m.bias)
                elif isinstance(m, nn.BatchNorm1d):
                    nn.init.ones_(m.weight)
                    nn.init.zeros_(m.bias)

        self.arcface.s = 8.0

    def forward(self, x, labels=None):
        # Извлекаем фичи
        features = self.backbone(x)

        # Получаем эмбеддинги
        embeddings = self.embedding(features)

        # Нормализуем для ArcFace (важно!)
        embeddings = F.normalize(embeddings, p=2, dim=1)

        if labels is not None:
            # Для обучения: возвращаем logits и эмбеддинги
            logits = self.arcface(embeddings, labels)
            return logits, embeddings

        # Для инференса: возвращаем только эмбеддинги
        return embeddings

# ==================== УТИЛИТЫ ДЛЯ ВАЛИДАЦИИ ====================
def analyze_embeddings(model, loader, device, num_samples=1000):
    """Анализ качества эмбеддингов"""
    model.eval()
    all_embeddings = []
    all_labels = []

    print("\nАнализ качества эмбеддингов...")
    with torch.no_grad():
        for i, (images, labels) in enumerate(tqdm(loader, desc="Extracting embeddings")):
            if i * loader.batch_size >= num_samples:
                break

            images = images.to(device)
            embeddings = model(images)  # Эмбеддинги уже нормализованы

            all_embeddings.append(embeddings.cpu())
            all_labels.append(labels)

    if not all_embeddings:
        return None, None

    all_embeddings = torch.cat(all_embeddings)
    all_labels = torch.cat(all_labels)

    # Берем подвыборку для анализа
    if len(all_embeddings) > 5000:
        indices = torch.randperm(len(all_embeddings))[:5000]
        all_embeddings = all_embeddings[indices]
        all_labels = all_labels[indices]

    # Вычисляем косинусные расстояния
    embeddings_norm = all_embeddings  # Уже нормализованы
    cosine_sim = torch.mm(embeddings_norm, embeddings_norm.t())

    # Анализ внутриклассовых и межклассовых расстояний
    intra_distances = []
    inter_distances = []

    unique_labels = torch.unique(all_labels)

    for i, label_i in enumerate(unique_labels[:20]):  # Ограничим для скорости
        mask_i = (all_labels == label_i)
        indices_i = torch.where(mask_i)[0]

        if len(indices_i) > 1:
            # Внутри класса
            for j in range(min(len(indices_i), 5)):  # Берем несколько пар
                for k in range(j+1, min(len(indices_i), 5)):
                    sim = cosine_sim[indices_i[j], indices_i[k]].item()
                    intra_distances.append(sim)

        # Между классами (берем первый другой класс)
        if i < len(unique_labels) - 1:
            label_j = unique_labels[i+1]
            mask_j = (all_labels == label_j)
            indices_j = torch.where(mask_j)[0]

            if len(indices_j) > 0:
                # Берем несколько пар между классами
                for _ in range(min(5, len(indices_i))):
                    idx_i = indices_i[torch.randint(0, len(indices_i), (1,))]
                    idx_j = indices_j[torch.randint(0, len(indices_j), (1,))]
                    sim = cosine_sim[idx_i, idx_j].item()
                    inter_distances.append(sim)

    if intra_distances and inter_distances:
        intra_mean = np.mean(intra_distances)
        inter_mean = np.mean(inter_distances)

        print(f"\n📊 Качество эмбеддингов:")
        print(f"  Средняя косинусная схожесть внутри класса: {intra_mean:.4f}")
        print(f"  Средняя косинусная схожесть между классами: {inter_mean:.4f}")
        print(f"  Ratio (inter/intra): {inter_mean/intra_mean if intra_mean > 0 else float('inf'):.2f}")

        # Создаем гистограмму расстояний
        fig, axes = plt.subplots(1, 2, figsize=(12, 5))

        # Гистограмма внутриклассовых расстояний
        axes[0].hist(intra_distances, bins=30, alpha=0.7, color='blue', density=True)
        axes[0].axvline(x=intra_mean, color='red', linestyle='--', label=f'Mean: {intra_mean:.3f}')
        axes[0].set_xlabel('Cosine Similarity (Intra-class)')
        axes[0].set_ylabel('Density')
        axes[0].set_title('Intra-class Similarity Distribution')
        axes[0].legend()
        axes[0].grid(True, alpha=0.3)

        # Гистограмма межклассовых расстояний
        axes[1].hist(inter_distances, bins=30, alpha=0.7, color='green', density=True)
        axes[1].axvline(x=inter_mean, color='red', linestyle='--', label=f'Mean: {inter_mean:.3f}')
        axes[1].set_xlabel('Cosine Similarity (Inter-class)')
        axes[1].set_ylabel('Density')
        axes[1].set_title('Inter-class Similarity Distribution')
        axes[1].legend()
        axes[1].grid(True, alpha=0.3)

        plt.tight_layout()
        plt.savefig('embedding_analysis.png', dpi=150, bbox_inches='tight')
        plt.show()

    return all_embeddings, all_labels

# ==================== ФУНКЦИИ ОБУЧЕНИЯ ====================
def train_epoch_ce(model, loader, optimizer, criterion, device):
    """Обучение CE модели для одной эпохи"""
    model.train()
    total_loss = 0
    correct = 0
    total = 0

    pbar = tqdm(loader, desc="Training CE")
    for images, labels in pbar:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        logits, _ = model(images)
        loss = criterion(logits, labels)
        loss.backward()

        # Gradient clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

        optimizer.step()

        total_loss += loss.item() * images.size(0)
        _, predicted = logits.max(1)
        correct += predicted.eq(labels).sum().item()
        total += images.size(0)

        current_acc = 100. * predicted.eq(labels).sum().item() / images.size(0)
        pbar.set_postfix({
            'loss': f'{loss.item():.4f}',
            'acc': f'{current_acc:.1f}%'
        })

    avg_loss = total_loss / total if total > 0 else 0
    accuracy = 100. * correct / total if total > 0 else 0

    return avg_loss, accuracy

def train_epoch_arcface(model, loader, optimizer, criterion, device, epoch, total_epochs):
    """Обучение ArcFace модели для одной эпохи"""
    model.train()
    total_loss = 0
    correct = 0
    total = 0

    # Progressive scaling - более плавное увеличение
    min_scale, max_scale = 8.0, 32.0
    if total_epochs <= 1:
        current_scale = max_scale
    else:
        progress = epoch / (total_epochs - 1)
        # Используем квадратичное увеличение
        current_scale = min_scale + (max_scale - min_scale) * (progress ** 0.5)

    model.arcface.s = current_scale

    pbar = tqdm(loader, desc=f"Training ArcFace (s={model.arcface.s:.1f}, m={model.arcface.m:.2f})")
    for batch_idx, (images, labels) in enumerate(pbar):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        logits, embeddings = model(images, labels)

        # DEBUG информация
        if batch_idx == 0 and epoch == 0:
            print(f"\n[DEBUG] Первый батч:")
            print(f"  Logits shape: {logits.shape}")
            print(f"  Logits range: [{logits.min():.4f}, {logits.max():.4f}]")
            print(f"  Logits mean/std: {logits.mean():.4f}/{logits.std():.4f}")
            print(f"  Embeddings norm: {embeddings.norm(p=2, dim=1).mean():.4f} ± {embeddings.norm(p=2, dim=1).std():.4f}")
            print(f"  Scale s: {model.arcface.s}")

        loss = criterion(logits, labels)
        loss.backward()

        # Gradient clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

        optimizer.step()

        # Вычисляем accuracy
        with torch.no_grad():
            # Для ArcFace используем cosine similarity
            embeddings_norm = embeddings  # Уже нормализованы
            weights_norm = F.normalize(model.arcface.weight.data, p=2, dim=1)
            cosine = torch.mm(embeddings_norm, weights_norm.t())
            predicted = cosine.argmax(dim=1)

            total_loss += loss.item() * images.size(0)
            correct += predicted.eq(labels).sum().item()
            total += images.size(0)

            current_acc = 100. * predicted.eq(labels).sum().item() / images.size(0)
            pbar.set_postfix({
                'loss': f'{loss.item():.4f}',
                'acc': f'{current_acc:.1f}%',
                'scale': f'{model.arcface.s:.1f}'
            })

    avg_loss = total_loss / total if total > 0 else 0
    accuracy = 100. * correct / total if total > 0 else 0

    return avg_loss, accuracy

def validate(model, loader, criterion, device, model_type='ce'):
    """Валидация модели"""
    model.eval()
    val_loss = 0
    val_correct = 0
    val_total = 0

    with torch.no_grad():
        pbar = tqdm(loader, desc="Validation")
        for images, labels in pbar:
            images, labels = images.to(device), labels.to(device)

            if model_type == 'ce':
                logits, _ = model(images)
                _, predicted = logits.max(1)
            else:  # arcface
                embeddings = model(images)
                embeddings_norm = embeddings  # Уже нормализованы
                weights_norm = F.normalize(model.arcface.weight.data, p=2, dim=1)
                cosine = torch.mm(embeddings_norm, weights_norm.t())
                logits = model.arcface.s * cosine
                predicted = cosine.argmax(dim=1)

            loss = criterion(logits, labels)
            val_loss += loss.item() * images.size(0)
            val_correct += predicted.eq(labels).sum().item()
            val_total += images.size(0)

            current_acc = 100. * predicted.eq(labels).sum().item() / images.size(0)
            pbar.set_postfix({'acc': f'{current_acc:.1f}%'})

    avg_loss = val_loss / val_total if val_total > 0 else 0
    accuracy = 100. * val_correct / val_total if val_total > 0 else 0

    return avg_loss, accuracy

# ==================== ОСНОВНАЯ ФУНКЦИЯ ====================
def main():
    # Настройка логирования
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    logging.basicConfig(
        level=logging.INFO,
        format='%(asctime)s - %(levelname)s - %(message)s',
        handlers=[
            logging.FileHandler(f'training_{timestamp}.log'),
            logging.StreamHandler()
        ]
    )
    logger = logging.getLogger(__name__)

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")
    if torch.cuda.is_available():
        print(f"GPU: {torch.cuda.get_device_name(0)}")
        print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")

    # Инициализация конфигурации
    config = Config()

    # ==================== 1. ПОДГОТОВКА ДАННЫХ ====================
    print("\n" + "="*60)
    print("ПОДГОТОВКА ДАННЫХ")
    print("="*60)

    # Здесь нужно загрузить ваши данные
    # config.identity_df = df_id  # ваш DataFrame
    # config.img_dir = img_dir    # путь к изображениям

    if not hasattr(config, 'identity_df') or config.identity_df is None:
        print("❌ Ошибка: identity_df не задан в конфигурации")
        print("Установите config.identity_df = ваш_dataframe")
        return

    if not hasattr(config, 'img_dir') or config.img_dir is None:
        print("❌ Ошибка: img_dir не задан в конфигурации")
        print("Установите config.img_dir = 'путь/к/изображениям'")
        return

    processor = CelebADataProcessor(config)
    num_classes = processor.filter_data()
    stats = processor.split_data_by_images()

    # Трансформации
    train_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomRotation(10),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                           std=[0.229, 0.224, 0.225])
    ])

    val_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                           std=[0.229, 0.224, 0.225])
    ])

    # Датасеты
    train_dataset = CelebAClassificationDataset(
        processor.train_df, config.img_dir, train_transform
    )
    val_dataset = CelebAClassificationDataset(
        processor.val_df, config.img_dir, val_transform
    )
    test_dataset = CelebAClassificationDataset(
        processor.test_df, config.img_dir, val_transform
    )

    # Даталоадеры
    train_loader = DataLoader(
        train_dataset, batch_size=config.batch_size,
        shuffle=True, num_workers=config.num_workers,
        pin_memory=True, drop_last=True
    )
    val_loader = DataLoader(
        val_dataset, batch_size=config.batch_size,
        shuffle=False, num_workers=config.num_workers
    )
    test_loader = DataLoader(
        test_dataset, batch_size=config.batch_size,
        shuffle=False, num_workers=config.num_workers
    )

    # Создаем папку для чекпоинтов
    os.makedirs('checkpoints', exist_ok=True)

    # ==================== 2. ОБУЧЕНИЕ CE МОДЕЛИ ====================
    print("\n" + "="*60)
    print("ОБУЧЕНИЕ CE МОДЕЛИ")
    print("="*60)

    ce_checkpoint_path = 'checkpoints/best_model_ce.pth'
    ce_model_exists = os.path.exists(ce_checkpoint_path)

    if not ce_model_exists:
        print("Обучение CE модели с нуля...")

        model_ce = SimpleFaceModel(
            num_classes=num_classes,
            embedding_size=config.embedding_size
        ).to(device)

        # Размораживаем больше слоев
        for param in model_ce.backbone.layer2.parameters():
            param.requires_grad = True
        for param in model_ce.backbone.layer3.parameters():
            param.requires_grad = True

        optimizer_ce = optim.AdamW(
            filter(lambda p: p.requires_grad, model_ce.parameters()),
            lr=0.001,
            weight_decay=1e-4
        )

        scheduler_ce = optim.lr_scheduler.CosineAnnealingLR(
            optimizer_ce,
            T_max=config.num_epochs_ce,
            eta_min=1e-6
        )

        criterion_ce = nn.CrossEntropyLoss(label_smoothing=0.1)

        best_ce_acc = 0
        ce_history = {'loss': [], 'acc': [], 'val_loss': [], 'val_acc': []}

        for epoch in range(config.num_epochs_ce):
            print(f"\nEpoch {epoch+1}/{config.num_epochs_ce} (CE)")
            print("-" * 40)

            train_loss, train_acc = train_epoch_ce(
                model_ce, train_loader, optimizer_ce, criterion_ce, device
            )

            val_loss, val_acc = validate(
                model_ce, val_loader, criterion_ce, device, 'ce'
            )

            # Сохраняем историю
            ce_history['loss'].append(train_loss)
            ce_history['acc'].append(train_acc)
            ce_history['val_loss'].append(val_loss)
            ce_history['val_acc'].append(val_acc)

            print(f"\nИтоги эпохи:")
            print(f"  Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%")
            print(f"  Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%")
            print(f"  Learning Rate: {optimizer_ce.param_groups[0]['lr']:.2e}")

            if val_acc > best_ce_acc:
                best_ce_acc = val_acc
                torch.save({
                    'epoch': epoch,
                    'model_state_dict': model_ce.state_dict(),
                    'val_acc': val_acc,
                    'optimizer_state_dict': optimizer_ce.state_dict(),
                    'history': ce_history,
                    'num_classes': num_classes
                }, ce_checkpoint_path)
                print(f"  ✓ Сохранена CE модель (Acc: {val_acc:.2f}%)")

            scheduler_ce.step()

        print(f"\n🎯 Лучшая точность CE модели: {best_ce_acc:.2f}%")
    else:
        print(f"✅ CE модель уже обучена: {ce_checkpoint_path}")

    # ==================== 3. FINE-TUNING С ARCFACE ====================
    print("\n" + "="*60)
    print("FINE-TUNING С ARCFACE LOSS")
    print("="*60)

    # Создаем ArcFace модель
    model_arcface = ArcFaceModel(
        num_classes=num_classes,
        embedding_size=config.embedding_size,
        s=8.0,  # Начинаем с меньшего scale
        m=config.arcface_m
    ).to(device)

    # Загружаем веса из CE модели
    print("\nЗагрузка весов из CE модели...")
    load_success = model_arcface.load_from_ce_model(ce_checkpoint_path, device)

    # Размораживаем все слои для fine-tuning
    for param in model_arcface.parameters():
        param.requires_grad = True

    # Проверка модели
    print("\n" + "="*50)
    print("ПРОВЕРКА МОДЕЛИ")
    print("="*50)

    model_arcface.eval()
    with torch.no_grad():
        test_images, test_labels = next(iter(train_loader))
        test_images = test_images[:4].to(device)
        test_labels = test_labels[:4].to(device)

        logits, embeddings = model_arcface(test_images, test_labels)

        print(f"\n1. Размерности:")
        print(f"   Logits shape: {logits.shape}")
        print(f"   Embeddings shape: {embeddings.shape}")

        print(f"\n2. Численные значения:")
        print(f"   Logits range: [{logits.min():.4f}, {logits.max():.4f}]")
        print(f"   Logits mean/std: {logits.mean():.4f} / {logits.std():.4f}")
        print(f"   Embeddings norm: {embeddings.norm(p=2, dim=1).mean():.4f} ± {embeddings.norm(p=2, dim=1).std():.4f}")

        print(f"\n3. Проверка accuracy:")
        embeddings_norm = embeddings  # Уже нормализованы
        weights_norm = F.normalize(model_arcface.arcface.weight.data, p=2, dim=1)
        cosine = torch.mm(embeddings_norm, weights_norm.t())
        predicted = cosine.argmax(dim=1)
        accuracy = (predicted == test_labels).float().mean().item()
        print(f"   Accuracy на тестовом батче: {accuracy:.2%}")

        test_loss = nn.CrossEntropyLoss()(logits, test_labels)
        print(f"   Loss: {test_loss.item():.4f}")

    # Оптимизатор для fine-tuning
    optimizer_arc = optim.AdamW(
        model_arcface.parameters(),
        lr=config.learning_rate,
        weight_decay=0.0005
    )

    # Scheduler с warmup
    scheduler_arc = optim.lr_scheduler.OneCycleLR(
        optimizer_arc,
        max_lr=config.learning_rate * 10,
        epochs=config.num_epochs_arcface,
        steps_per_epoch=len(train_loader),
        pct_start=0.3
    )

    # Loss function с label smoothing
    criterion_arc = nn.CrossEntropyLoss(label_smoothing=0.1)

    # Обучение ArcFace
    print("\n" + "="*60)
    print("НАЧАЛО ОБУЧЕНИЯ ARCFACE")
    print("="*60)

    best_arc_acc = 0
    arc_history = {'loss': [], 'acc': [], 'val_loss': [], 'val_acc': [], 'scale': []}

    for epoch in range(config.num_epochs_arcface):
        print(f"\n{'='*50}")
        print(f"Epoch {epoch+1}/{config.num_epochs_arcface} - ArcFace Fine-tuning")
        print(f"{'='*50}")

        # Обучение
        train_loss, train_acc = train_epoch_arcface(
            model_arcface, train_loader, optimizer_arc, criterion_arc, device,
            epoch, config.num_epochs_arcface
        )

        # Валидация
        val_loss, val_acc = validate(
            model_arcface, val_loader, criterion_arc, device, 'arcface'
        )

        # Сохраняем историю
        arc_history['loss'].append(train_loss)
        arc_history['acc'].append(train_acc)
        arc_history['val_loss'].append(val_loss)
        arc_history['val_acc'].append(val_acc)
        arc_history['scale'].append(model_arcface.arcface.s)

        print(f"\n📊 Итоги эпохи:")
        print(f"  Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%")
        print(f"  Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%")
        print(f"  Scale s: {model_arcface.arcface.s:.1f}")
        print(f"  Margin m: {model_arcface.arcface.m:.2f}")
        print(f"  Learning Rate: {optimizer_arc.param_groups[0]['lr']:.2e}")

        # Сохраняем лучшую модель
        if val_acc > best_arc_acc:
            best_arc_acc = val_acc
            torch.save({
                'epoch': epoch,
                'model_state_dict': model_arcface.state_dict(),
                'optimizer_state_dict': optimizer_arc.state_dict(),
                'val_acc': val_acc,
                'scale': model_arcface.arcface.s,
                'margin': model_arcface.arcface.m,
                'history': arc_history,
                'num_classes': num_classes
            }, 'checkpoints/best_model_arcface.pth')
            print(f"  💾 Сохранена лучшая ArcFace модель (Val Acc: {val_acc:.2f}%)")

        # Обновляем scheduler
        scheduler_arc.step()

    # ==================== 4. ТЕСТИРОВАНИЕ ====================
    print("\n" + "="*60)
    print("ТЕСТИРОВАНИЕ ARCFACE МОДЕЛИ")
    print("="*60)

    # Загружаем лучшую ArcFace модель
    arc_checkpoint_path = 'checkpoints/best_model_arcface.pth'
    if os.path.exists(arc_checkpoint_path):
        arc_checkpoint = torch.load(arc_checkpoint_path, map_location=device)
        model_arcface.load_state_dict(arc_checkpoint['model_state_dict'])
        print(f"✅ Загружена лучшая ArcFace модель (Val Acc: {arc_checkpoint['val_acc']:.2f}%)")
    else:
        print("⚠ Лучшая модель не найдена, используем последнюю")

    # Тестирование
    test_loss, test_acc = validate(
        model_arcface, test_loader, criterion_arc, device, 'arcface'
    )

    print(f"\n🎯 РЕЗУЛЬТАТЫ ARCFACE НА ТЕСТОВОЙ ВЫБОРКЕ:")
    print(f"  Test Loss: {test_loss:.4f}")
    print(f"  Test Accuracy: {test_acc:.2f}%")
    print(f"  Final Scale s: {model_arcface.arcface.s:.1f}")
    print(f"  Final Margin m: {model_arcface.arcface.m:.2f}")

    # Анализ эмбеддингов
    print("\n" + "="*60)
    print("АНАЛИЗ КАЧЕСТВА ЭМБЕДДИНГОВ")
    print("="*60)

    test_embeddings, test_labels = analyze_embeddings(
        model_arcface, test_loader, device, num_samples=2000
    )

    # ==================== 5. ВИЗУАЛИЗАЦИЯ ====================
    print("\n" + "="*60)
    print("ВИЗУАЛИЗАЦИЯ РЕЗУЛЬТАТОВ ОБУЧЕНИЯ")
    print("="*60)

    # Создаем графики
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))

    # Loss history
    axes[0, 0].plot(arc_history['loss'], label='Train Loss', linewidth=2, marker='o', markersize=4)
    axes[0, 0].plot(arc_history['val_loss'], label='Val Loss', linewidth=2, marker='s', markersize=4)
    axes[0, 0].set_xlabel('Epoch')
    axes[0, 0].set_ylabel('Loss')
    axes[0, 0].set_title('ArcFace: Loss History')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)

    # Accuracy history
    axes[0, 1].plot(arc_history['acc'], label='Train Acc', linewidth=2, marker='o', markersize=4)
    axes[0, 1].plot(arc_history['val_acc'], label='Val Acc', linewidth=2, marker='s', markersize=4)
    axes[0, 1].set_xlabel('Epoch')
    axes[0, 1].set_ylabel('Accuracy (%)')
    axes[0, 1].set_title('ArcFace: Accuracy History')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)
    axes[0, 1].set_ylim([0, 100])

    # Scale s history
    axes[1, 0].plot(arc_history['scale'], linewidth=2, marker='o', markersize=4, color='green')
    axes[1, 0].fill_between(range(len(arc_history['scale'])),
                           arc_history['scale'],
                           alpha=0.3, color='green')
    axes[1, 0].set_xlabel('Epoch')
    axes[1, 0].set_ylabel('Scale s')
    axes[1, 0].set_title('ArcFace Scale Progression')
    axes[1, 0].grid(True, alpha=0.3)

    # Learning rate history (симулируем)
    lr_values = []
    temp_optimizer = optim.AdamW([torch.zeros(1)], lr=config.learning_rate)
    temp_scheduler = optim.lr_scheduler.OneCycleLR(
        temp_optimizer,
        max_lr=config.learning_rate * 10,
        epochs=config.num_epochs_arcface,
        steps_per_epoch=len(train_loader),
        pct_start=0.3
    )

    for epoch in range(config.num_epochs_arcface):
        for _ in range(len(train_loader)):
            lr_values.append(temp_optimizer.param_groups[0]['lr'])
            temp_scheduler.step()

    # Берем значения на начало каждой эпохи
    epoch_lr = [lr_values[i * len(train_loader)] for i in range(config.num_epochs_arcface)]

    axes[1, 1].plot(epoch_lr, linewidth=2, marker='o', markersize=4, color='orange')
    axes[1, 1].set_xlabel('Epoch')
    axes[1, 1].set_ylabel('Learning Rate')
    axes[1, 1].set_title('Learning Rate Schedule (OneCycle)')
    axes[1, 1].grid(True, alpha=0.3)
    axes[1, 1].set_yscale('log')

    plt.tight_layout()
    plt.savefig(f'arcface_training_results_{timestamp}.png', dpi=150, bbox_inches='tight')
    plt.show()

    print("\n" + "="*60)
    print("🎉 ОБУЧЕНИЕ ARCFACE ЗАВЕРШЕНО!")
    print("="*60)
    print(f"📊 Итоговые результаты:")
    print(f"  Лучшая точность на валидации: {best_arc_acc:.2f}%")
    print(f"  Точность на тесте: {test_acc:.2f}%")
    print(f"  Финальный Scale s: {model_arcface.arcface.s:.1f}")
    print(f"  Финальный Margin m: {model_arcface.arcface.m:.2f}")
    print(f"\n💾 Файлы сохранены:")
    print(f"  Чекпоинты: 'checkpoints/'")
    print(f"  Лог: 'training_{timestamp}.log'")
    print(f"  Графики: 'arcface_training_results_{timestamp}.png'")
    print(f"  Анализ эмбеддингов: 'embedding_analysis.png'")

if __name__ == "__main__":
    main()